In [1]:
import mdtraj as md
import numpy as np

In [2]:
# inputdir, outputdir
inputdir = "input"

outputdir = "output"

In [3]:
trj = md.load_trr(f"{inputdir}/out.trr", top=f"{inputdir}/solv_ions_prot.gro")
print(trj)

<mdtraj.Trajectory with 500 frames, 4792 atoms, 309 residues, and unitcells>


# make weight list

In [4]:
from weightdict import atomicWeightsDecimal as wdict
weight_key = 'standard'

atoms_dict = trj.topology.to_dataframe()[0].element

wlist = [float(wdict[atom][weight_key]) for atom in atoms_dict]

In [5]:
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

146084.19282618037
146843.58345148605
148134.37109819264
147364.9272199925
146850.12732746112
146612.7168842983
145752.69593981758
144452.36391155253
144152.75907437803
145286.57522874922


# centering

In [6]:
trj.center_coordinates(mass_weighted=True)

<mdtraj.Trajectory with 500 frames, 4792 atoms, 309 residues, and unitcells at 0x1100de080>

In [7]:
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

-0.3805274908356324
-0.38100691824377364
-0.3695024619539815
-0.37476771859666513
-0.38466225700586776
-0.3857457324228655
-0.38348396196329304
-0.3878871178540919
-0.3793476893772443
-0.37027075142835386


# choose 2 frames

In [8]:
a = trj.xyz[0]
b = trj.xyz[-1]

# matrix U

In [9]:
U = np.empty((3,3))

for i in range(3):
    for j in range(3):
        U[i][j] = sum( [wlist[k]*a[k ,i]*b[k, j] for k in range(a.shape[0])] )

print(U)

[[ 62914.81288118  38547.79777827 -33233.75318807]
 [ -7875.27131628  37121.66933133  -5875.02421762]
 [ 13060.44571078   8975.06224838  14449.49038106]]
